# FM(Factorized Machine)因子分解机

该方法常用的场景是推荐，在snippets-rec中会有FM及其衍生算法的介绍。从ml的角度看，该算法不仅仅是一个回归算法, 也可以用于分类、排序等等。

## 1.1 简介
FM可以视为线性回归引入特征交叉的一种演进:

1)线性回归引入特征交叉
$$
\hat{y}(x)=w_0 + \underbrace{\sum_{i=1}^p w_i x_i}_{\text {线性回归 }}+\underbrace{\sum_{i=1}^p \sum_{j=i+1}^p w_{i j} x_i x_j}_{\text {交叉项 (组合待征) }}
$$

其中样本$x_i$为样本x的第$i$维特征，$p$为维数.
但在当x为稀疏数据时（如商品类别或标签）维度可能有几万甚至上百万。交叉项会有维度爆炸，同时绝大多数维度是不相关的，导致交叉项非常稀疏，很难学习。因此FM通过矩阵分解对矩阵$V$进行了近似.

2)FM: 通过矩阵分解对交叉项进行近似
$$
\begin{aligned}
交叉相部分= \\
& \sum_{i=1}^{p} \sum_{j=i+1}^p\left\langle\mathbf{v}_i, \mathbf{v}_j\right\rangle x_i x_j \\
= & \frac{1}{2} \sum_{i=1}^p \sum_{j=1}^p\left\langle\mathbf{v}_i, \mathbf{v}_j\right\rangle x_i x_j-\frac{1}{2} \sum_{i=1}^p\left\langle\mathbf{v}_i, \mathbf{v}_i\right\rangle x_i x_i \\
= & \frac{1}{2}\left(\sum_{i=1}^p \sum_{j=1}^p \sum_{f=1}^k v_{i, f} v_{j, f} x_i x_j-\sum_{i=1}^p \sum_{f=1}^k v_{i, f} v_{i, f} x_i x_i\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left(\sum_{i=1}^n v_{i, f} x_i\right)\left(\sum_{j=1}^n v_{j, f} x_j\right)-\sum_{i=1}^n v_{i, f}^2 x_i^{2}\right) \\
= & \frac{1}{2} \sum_{f=1}^k\left(\left ( \sum_{i=1}^n v_{i, f} x_i \right)^{2}-\sum_{i=1}^n v_{i, f}^2 x_i^{2}\right)
\end{aligned}
$$

矩阵表示:
$$
\sum_{i=1}^{n} \sum_{j=i+1}^{n} \langle \textbf v_i, \textbf v_j \rangle x_i x_j =
\frac{1}{2} \sum_{f=1}^{k} \Big( \big(\sum_{i=1}^{n} v_f^{(i)} x_i \big)^2 - \sum_{i=1}^{n}v_f^{(i) 2} x_i^2 \Big) =
\frac{1}{2} \sum_{f=1}^{} \Big( S_{1,f}^2 - S_{2,f} \Big) =
\frac{1}{2} \Big( S_{1}^2 - S_{2} \Big)
$$
其中:
- $S_{1}$表示$VX$沿第一个维度求和: $S_1 = \sum_{i=1}^{n} v_f^{(i)} x_i $


In [ ]:
import torch


class TorchFM(torch.nn.Module):
    def __init__(self, n=None, k=None):
        super().__init__()
        self.V = torch.nn.Parameter(torch.randn(n, k), requires_grad=True)
        self.linear = torch.nn.Linear(n, 1)

    def forward(self, x):
        out_1 = torch.matmul(x, self.V).pow(2).sum(1, keepdim=True)  #S_1^2
        out_2 = torch.matmul(x.pow(2), self.V.pow(2)).sum(1, keepdim=True)  # S_2

        out_inter = 0.5 * (out_1 - out_2)
        out_linear = self.linear(x)
        out = out_inter + out_linear

        return out

In [2]:
import torch
vv = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [3]:
vv.pow(2)

tensor([[ 1,  4,  9],
        [16, 25, 36],
        [49, 64, 81]])